# **Configurando as bibliotecas do projeto**

## *Instalação de bibliotecas*

## *Importação de bibliotecas*

In [1]:
'''
    Essa célula será usada para a importação de bibliotecas.
'''

# Instala a biblioteca que será utilizada para se obter os dados financeiros.
import yfinance as yf
# Importa a biblioteca que será utilizada para a manipulação de DataFrames.
import pandas as pd
# Importa a biblioteca que será utilizada para lidar com objetos do tipo "datetime".
from datetime import datetime, timedelta
# Importa a biblioteca que será responsável pela tipagem de funções.
from typing import Union, Optional
# Importa a biblioteca que será utilizada para a manipulação de arrays e também para o uso de algumas funções matemáticas.
import numpy as np

# **Data Engineering**

### *Configurações iniciais*

In [ ]:
'''
    Essa célula será usada para se obter os tickets das companhias que fazem parte do S&P 500, com base em dados da wikipedia.
'''

# Salva em uma variável a url que contém a tabela com os tickets das companhias.
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Lê todas as tabelas presentes na url acima.
sp500_table = pd.read_html(url)

# Salva a coluna "Symbol" da primeira tabela em uma variável (tal coluna contém todos os tickets das companhias que fazem parte do S&P 500).
sp500_tickets = sp500_table[0]['Symbol']

# Transforma os tickets obtidos em uma lista.
sp500_tickets = sp500_tickets.tolist()

# Exibe a lista de tickets criada acima.
sp500_tickets

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BK',
 'BBWI',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG'

In [ ]:
'''
    Essa célula será usada para definir o intervalo de tempo dos dados que serão coletados, especificando a data de início e a data de fim.
'''

# Define a data inicial cujos dados serão coletados.
start_date = datetime(2019,1,2).date()
# Define a data final cujos dados serão coletados.
end_date = datetime(2023,12,30).date()

In [ ]:
'''
    Essa célula será usada para criar um dicionário chamado setup, que contém os parâmetros principais para a coleta de dados, 
    incluindo a lista de tickers das companhias do S&P 500 e o intervalo de tempo.
'''

# Cria um dicionário para guardar alguns parâmetros importantes para a coleta de dados.
setup = {
    "tickets": sp500_tickets,
    "start_date": start_date,
    "end_date": end_date
}

### *Obtendo dados dos tickers via yfinance (Open, High, Low, Close, Adj Close, Volume)*

In [ ]:
'''
    Essa célula será usada para a obtenção de alguns dados (Open, High, Low, Close, Adj Close, Volume) 
    dos tickers presentes na variável de configuração setup.
'''

# Cria um array que guardará os DataFrames que contém os dados referentes a cada um dos tickers presentes na variável de configuração setup.
data_array = np.empty(len(sp500_tickets), dtype=object)

# Itera sobre cada um dos tickers presentes na variável de configuração setup.
for i, ticker in enumerate(sp500_tickets):
    # Faz o download dos dados do ticker em questão
    ticker_data = yf.download(ticker, setup['start_date'], setup['end_date'])
    # Adiciona um nome ao DataFrame do ticker em questão.
    ticker_data.name=ticker
    # Salva o DataFrame do ticker em questão na variável "data_array".
    data_array[i] = ticker_data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

$BF.B: possibly delisted; No price data found  (1d 2019-01-02 -> 2023-12-30)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

### *Filtrando os dados obtidos*

In [ ]:
'''
    Essa célula será responsável por filtrar a variável "data_array" de modo que esta só contenha DataFrames não vazios.
'''

# Cria um filtro para identificar quais DataFrames presentes em "data_array" são vazios, isto é, não possuem nenhum índice.
void_filter = np.array([df.index.empty for df in data_array])

# Aplica a negação do filtro gerado acima (ou seja, filtra os DataFrames que não são vazios) no array "data_array" e armazena o resultado 
# na variável "filtered_data_array". 
filtered_data_array = data_array[~void_filter]

# Exibe uma mensagem que informa quantos tickers foram removidos na filtragem realizada acima.
print(f"Foram removidos {len(data_array) - len(filtered_data_array)} tickers que estavam vazios.")

Foram removidos 5 tickers que estavam vazios.


In [ ]:
'''
    Essa célula será responsável por filtrar a variável "data_array" de modo que esta só contenha DataFrames cujos índices se iniciem em 
    setup['start_date'].
'''

# Armazena o tamanho atual do array "filtered_data_array" para que se tenha ideia ao final dessa célula, de quantos tickers 
# foram removidos com o filtro que será aplicado aqui.
filtered_data_array_initial_len = len(filtered_data_array)

# Cria um filtro para saber quais dos DataFrames presentes em "filtered_data_array" possuem datas iniciais (isto é, índices de posição 0) 
# iguais à "setup['start_date']".
start_date_filter = [df.index[0].date() == setup['start_date'] for df in filtered_data_array]

# Aplica o filtro criado acima, removendo assim todos os DataFrames presentes em "filtered_data_array" cujos índices iniciais são diferentes de 
# "setup['start_date']".
filtered_data_array = filtered_data_array[start_date_filter]

# Exibe uma mensagem informando quantos DataFrames não satisfaziam a condição do filtro criado acima, e informando também quantos DataFrames
# ainda existem em "data_array".
print(f"Foram removidos {filtered_data_array_initial_len - len(filtered_data_array)} tickers que não possuiam data inicial igual à {setup['start_date']}. Agora, todos os demais {len(filtered_data_array)} tickers \npossuem data de inicio em {setup['start_date']}.")

Foram removidos 13 tickers que não possuiam data inicial igual à 2019-01-02. Agora, todos os demais 485 tickers 
possuem data de inicio em 2019-01-02.


In [ ]:
'''
    Essa célula será utilizada para mostrar que, nos dados baixados, não existem DataFrames em "data_array" cujos índices terminam em uma 
    data anterior à última data de negociação possível antes de setup['end_date'], isto é não existem DataFrames em "data_array" cujos índices
    terminam em uma data anterior à 2023-12-29.
'''

# Obtem qual é o menor dentre os últimos índices de cada um dos DataFrames presentes em "filtered_data_array"
lowest_last_date = np.min(np.array([df.index[len(df.index)-1] for df in filtered_data_array]))

# Exibe uma mensagem que informa qual é a menor data final entre todos os DataFrames presentes em "filtered_data_array" e gera uma
# conclusão a partir de tal data.
print(f"Dentre os tickers que passaram nos filtros acima, a menor data final é {lowest_last_date.date()}, que é a última data possível de negociação dentro do \nperiodo estabelecido na variável 'setup' ({setup['end_date']}). Ou seja, nenhum filtro precisa ser aplicado em relação a data final dos tickers.")

Dentre os tickers que passaram nos filtros acima, a menor data final é 2023-12-29, que é a última data possível de negociação dentro do 
periodo estabelecido na variável 'setup' (2023-12-30). Ou seja, nenhum filtro precisa ser aplicado em relação a data final dos tickers.
